# CSCD 429: HW2
## Colin Middleton
## Dr. Dan Li
## 10/21/20

# Outline
## Imports
## Read Data Into Dataframes
## Data Exploration
## Preprocessing

## Package Imports

In [201]:
import pandas as pd # For data manipulation
import numpy as np # For calculating things
from statistics import mode # For finding most frequent class label from nearest neighbors

## Read Data Into Dataframes

In [202]:
# 'Training' Data
data_Genes_relation = pd.read_csv("gene_files/Genes_relation.data")
data_Interactions_relation = pd.read_csv("gene_files/Interactions_relation.data", header=None)

# Testing Data
test_Genes_relation = pd.read_csv("gene_files/Genes_relation.test")
test_Interactions_relation = pd.read_csv("gene_files/Interactions_relation.test", header=None)
keys_relation = pd.read_csv("gene_files/keys.txt")

## Data Exploration

### Genes_relation

In [203]:
# Check data values
data_Genes_relation.head()

,GeneID,Essential,Class,Complex,Phenotype,Motif,Chromosome,Function,Localization
0,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00824,1,CELLULAR ORGANIZATION (proteins are localized ...,cytoplasm
1,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00824,1,PROTEIN SYNTHESIS,cytoplasm
2,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00825,1,CELLULAR ORGANIZATION (proteins are localized ...,cytoplasm
3,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00825,1,PROTEIN SYNTHESIS,cytoplasm
4,G234065,Non-Essential,ATPases,?,?,?,1,"CELL RESCUE, DEFENSE, CELL DEATH AND AGEING",cytoplasm


In [204]:
data_Genes_relation.describe()

,GeneID,Essential,Class,Complex,Phenotype,Motif,Chromosome,Function,Localization
count,4346,4346,4346,4346,4346,4346,4346,4346,4346
unique,862,4,24,52,13,236,17,13,15
top,G234227,Non-Essential,?,?,?,?,4,CELLULAR ORGANIZATION (proteins are localized ...,nucleus
freq,60,2824,2657,1890,1065,2239,672,1532,1671


### Interactions_relation

In [205]:
# Label columns in Interactions data and check values
data_Interactions_relation.columns = ['GeneID1', 'GeneID2', 'Type', 'Expression_Corr']
data_Interactions_relation.head()

,GeneID1,GeneID2,Type,Expression_Corr
0,G238510,G239467,Genetic,0.252653076.
1,G235550,G239467,Physical,0.709247942.
2,G235621,G239467,Physical,-0.001238791.
3,G235265,G239467,Physical,0.482255315.
4,G234935,G234445,Physical,-0.460856268.


In [206]:
data_Interactions_relation.describe()

,GeneID1,GeneID2,Type,Expression_Corr
count,910,910,910,910
unique,460,453,3,817
top,G235439,G237021,Physical,1.
freq,14,10,549,43


### test_Genes_relation

In [207]:
test_Genes_relation.head()

,GeneID,Essential,Class,Complex,Phenotype,Motif,Chromosome,Function,Localization
0,G234273,Essential,ATPases,Chaperonine containing T-complex TRiC (TCP RIN...,Cell morphology and organelle mutants,?,4,?,?
1,G234490,Non-Essential,Transcription factors,?,Cell morphology and organelle mutants,PS00028,8,?,?
2,G236380,?,?,?,?,?,7,?,?
3,G234394,Essential,Actins,Cytoskeleton,Cell cycle defects,PS00432,6,?,?
4,G234385,Non-Essential,Molecular chaperones,?,"Auxotrophies, carbon and",PS00190,6,?,?


### test_Interactions_relation

In [208]:
test_Interactions_relation.head()

,0,1,2,3
0,G234651,G235906,Physical,0.72595548.
1,G234565,G235541,Genetic-Physical,0.87259026.
2,G234565,G235681,?,0.49456466.
3,G234570,G235695,Physical,-0.05846251.
4,G234299,G234974,?,0.35732472.


### keys_relation

In [209]:
keys_relation.head()

,GeneID,Localization
0,G234062,mitochondria
1,G234081,nucleus
2,G234086,mitochondria
3,G234087,nucleus
4,G234094,nucleus


## Preprocessing

In [210]:
def preprocess(genes_relation, interactions_relation):
    # Step 1: Rename interactions_relation attributes
    interactions_relation_1 = interactions_relation.copy()
    interactions_relation_1.columns = ['GeneID1', 'Interaction', 'Type', 'Expression_Corr']
    # Step 2: Left join genes_relation and interactions_relation_1 on 'GeneID'
    combined_relation = genes_relation.join(interactions_relation_1.set_index('GeneID1'), on='GeneID', how='left')
    # Step 3: Drop 'Function' attribute
    combined_relation.drop(columns='Function', axis=1, inplace=True)
    # Step 4: Replaced all '?' and '?.' with NaN
    combined_relation.replace('?', np.nan, inplace=True)
    combined_relation.replace('?.', np.nan, inplace=True)

    return combined_relation

In [211]:
combined_training = preprocess(data_Genes_relation, data_Interactions_relation)
combined_training.head()

,GeneID,Essential,Class,Complex,Phenotype,Motif,Chromosome,Localization,Interaction,Type,Expression_Corr
0,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,NaN,PS00824,1,cytoplasm,G234126,Genetic-Physical,0.914095071.
0,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,NaN,PS00824,1,cytoplasm,G235065,Genetic-Physical,0.751584888.
1,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,NaN,PS00824,1,cytoplasm,G234126,Genetic-Physical,0.914095071.
1,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,NaN,PS00824,1,cytoplasm,G235065,Genetic-Physical,0.751584888.
2,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,NaN,PS00825,1,cytoplasm,G234126,Genetic-Physical,0.914095071.


In [212]:
combined_test = preprocess(test_Genes_relation, test_Interactions_relation)
# Remove 'Localization' attribute
combined_test.drop(columns='Localization', axis=1, inplace=True)
combined_test.head()

,GeneID,Essential,Class,Complex,Phenotype,Motif,Chromosome,Interaction,Type,Expression_Corr
0,G234273,Essential,ATPases,Chaperonine containing T-complex TRiC (TCP RIN...,Cell morphology and organelle mutants,NaN,4,G234779,Genetic,NaN
1,G234490,Non-Essential,Transcription factors,NaN,Cell morphology and organelle mutants,PS00028,8,G234575,NaN,-0.40745963.
2,G236380,NaN,NaN,NaN,NaN,NaN,7,G235332,Physical,0.82722864.
3,G234394,Essential,Actins,Cytoskeleton,Cell cycle defects,PS00432,6,G234215,Genetic,0.55772219.
3,G234394,Essential,Actins,Cytoskeleton,Cell cycle defects,PS00432,6,G235769,NaN,0.55519542.


# Find Entropy of each attribute

In [213]:
# Status: Working
def Info(attribute_column, attribute_value, class_column):
    # Calculate the information needed to classify a tuple of this attribute_value in D

    
    count_value_occurances = list(attribute_column).count(attribute_value)
    dataframe_subset = pd.concat([attribute_column, class_column], axis=1)
    dataframe_subset.columns = ['Attribute Value', 'Class Value']
    # print(dataframe_subset)

    # Only look at the tuples with attribute_column value == value
    dataframe_subset1 = dataframe_subset[dataframe_subset['Attribute Value'] == attribute_value]

    # print(dataframe_subset1)

    # Calculate -p_i log_2(p_i) for each class_value and sum them up
    inner_sum = 0
    for class_value in class_column.unique():
        # Count the number of occurances of a class (for this attribute_value)
        class_value_count = list(dataframe_subset1['Class Value']).count(class_value)
        # Determine p_i from class
        p_i = class_value_count / count_value_occurances

        # print('Class value: {}'.format(class_value))
        # print('p_i: {:1.4f}'.format(p_i))

        # Calculate Entropy and add them all up
        if p_i != 0:
            inner_sum = inner_sum - (p_i) * np.log2(p_i)
        # Ignore p_i if it is 0
        # This prevents us from taking the log of 0 (which is undefined)
    return inner_sum

In [214]:
# Status: Working
def Info_A(attribute_column, class_column):
    # Calculate the information needed to classify tuples after split on attribute A

    outer_sum = 0
    # Find the number of tuples in this attribute
    attribute_size = len(list(attribute_column))
    for value in attribute_column.unique():
        # Find |D_j| in our Information equation
        num_value_occurances = list(attribute_column).count(value)
        # Find Info_A (D) in our Information equation
        outer_sum = outer_sum + (num_value_occurances / attribute_size) * Info(attribute_column, value, class_column)
    return outer_sum

In [215]:
# Status: Working
def find_Entropies_of(dataframe, class_attribute, exclude_attribute):
    """
    Calculate the information needed to split on each attribute

    Input: 
        dataframe: DataFrame object with all data
        class_attribute: String specifying column of class we want to predict
    Return:
        A DataFrame object listing the information needed to split on each attribute (other than the class_attribute)

    Note: This algorithm skips over missing values
    Can handle when no occurances of a class value for a specific attribute value (when p_i == 0)
    """

    infos = pd.DataFrame({'Attribute': [], 'Entropy': []})
    dataframe_minus_class_attribute = dataframe.drop(class_attribute, axis=1)
    dataframe_minus_class_attribute.drop(exclude_attribute, axis=1, inplace=True)
    # print(dataframe_minus_class_attribute)

    # Get all Infos into 'infos' dataframe
    for attribute in dataframe_minus_class_attribute.columns:
        # For each attribute calculate the Info_A(D)
        new_row = {'Attribute': attribute, 'Entropy': Info_A(dataframe[attribute], dataframe[class_attribute])}
        infos = infos.append(new_row, ignore_index=True)

    # Sort attributes by decreasing Entropy of splitting on that attribute
    infos.sort_values(by='Entropy', axis=0, inplace=True)
    # Change the index to the attribute names
    infos.set_index('Attribute', drop=True, inplace=True)

    return infos

In [216]:
find_Entropies_of(combined_training, class_attribute='Localization', exclude_attribute='GeneID')

,Entropy
Attribute,
Expression_Corr,0.069748
Interaction,0.276533
Complex,0.301106
Motif,0.342933
Class,0.391346
Type,1.721958
Phenotype,1.763032
Chromosome,2.203626
Essential,2.431559


### Findings
From highest correlation (lowest expected entropy) with 'Localization' to lowest (highest expected entropy) we have:

1. Expression_Corr
1. Interaction
1. Complex
1. Motif
1. Class
1. Phenotype
1. Chromosome
1. Essential

# Model Building Methods
Using KNN just like class example: Hayashi

Except weighting based on entropy.

In [217]:
def find_weights(training_relation, class_attribute, id_attribute):
    # Calculate entropy for all attributes
    info_dataframe = find_Entropies_of(training_relation, class_attribute, id_attribute)
    weights_dataframe = info_dataframe.copy()

    # Calculate weights using 1/ entropy
    # This just reverses the magnitudes of the entropies, keeping everything positive
    for index, row in weights_dataframe.iterrows():
        if row['Entropy'] != 0:
            row['Entropy'] = 1 / row['Entropy']
    # weights_dataframe['Entropy'] = 1 / weights_dataframe['Entropy']
    weights_dataframe.rename(columns={'Entropy':'Weights'}, inplace=True)
    weights_dataframe = weights_dataframe.iloc[:4]
    return weights_dataframe

In [218]:
def similarity(weights, tuple_i, tuple_j):
    '''
    Input
        weights: DataFrame containing the list of attributes (as index values) and the associated weight of each attribute (as column)
        tuple_i: tuple with labeled columns
        tuple_j: tuple with labeled columns
    '''
    # Find distance from the interaction relation
    # dist_sum = distance_from_interaction(weights['Interaction'], interactions_dataframe, tuple_i, tuple_j)
    sim_sum = 0
    for attribute in weights.index:
        attribute_weight = float(weights['Weights'][attribute])
        # Only add similarity between tuples if attributes agree
        # Note: this does not add if one or both values are NaN
        if tuple_i[attribute] == tuple_j[attribute]:
            sim_sum = sim_sum + attribute_weight
            
    return sim_sum

In [219]:
def find_similarities(test_tuple, training_relation, id_attribute, weights):
    similarities_dataframe = pd.DataFrame({'From':[], 'To':[], 'Similarity':[]})

    for index, training_tuple in training_relation.iterrows():
        sim_ij = similarity(weights, test_tuple, training_tuple)
        row_to_add = pd.DataFrame({'From':[training_tuple[id_attribute]], 'To':[test_tuple[id_attribute]], 'Similarity':[sim_ij]})
        similarities_dataframe = similarities_dataframe.append(row_to_add.copy(), ignore_index=True)
    
    similarities_dataframe.set_index('From', drop=True, inplace=True)
    return similarities_dataframe

In [225]:
class MyKNNModel:
    def __init__(self, training_relation, class_attribute, id_attribute, k):
        # This is basically the training of the model
        # The only thing that is calculated here are the weights
        self.training_data = training_relation
        self.k = k
        self.class_attribute = class_attribute
        self.id_attribute = id_attribute
        self.weights = find_weights(self.training_data, self.class_attribute, self.id_attribute)
        self.predictions = pd.DataFrame({self.id_attribute:[], 'Prediction':[]}) # Create output dataframe to use later

    def predict(self, test_relation):
        # For each test tuple, calculate the distances between it and all the training tuples
        for index, test_tuple in test_relation.iterrows():
            similarities_to_tuple = find_similarities(test_tuple, self.training_data, self.id_attribute, self.weights)

            # Find the k nearest neighbors and record their class labels
            knn_labels = np.array([])
            while len(knn_labels) < self.k:
                # Find the nearest neighbor
                max_sim = max(similarities_to_tuple['Similarity'])
                max_sim_GeneID = similarities_to_tuple.loc[similarities_to_tuple['Similarity'] == max_sim].index[0]
                
                # Add that neighbor's class label to the list for voting
                label_to_add = np.array([self.training_data[self.training_data[self.id_attribute] == max_sim_GeneID][self.class_attribute]][0])
                knn_labels = np.concatenate((knn_labels, label_to_add))

                # Remove that tuple from list so next nearest can be found
                similarities_to_tuple.drop(max_sim_GeneID, inplace=True)

            # Now knn_labels has the labels of the k nearest neighbors to this particular test tuple
            # Find the most frequent label in knn_labels and set that as the prediction for this test_tuple
            self.predictions = self.predictions.append({self.id_attribute: test_tuple[self.id_attribute], 'Prediction': mode(knn_labels)}, ignore_index=True)

        return

# Build Model

## Choose smaller datasets: Original training was 7244 tuples, just take first 500 of those, same with test set

In [221]:
new_training = combined_training.iloc[:500]
new_test = combined_test.iloc[:500]
new_keys = keys_relation.iloc[:500]

In [226]:
my_model = MyKNNModel(new_training, class_attribute='Localization', id_attribute='GeneID', k=5)

In [227]:
my_model.weights

,Weights
Attribute,
Interaction,0.000000
Expression_Corr,0.000000
Complex,8.662915
Motif,4.277524


In [228]:
my_model.predict(new_test)

In [229]:
my_model.predictions.head()

,GeneID,Prediction
0,G234273,cytoplasm
1,G234490,cytoplasm
2,G236380,cytoplasm
3,G234394,cytoskeleton
4,G234394,cytoskeleton


In [176]:
new_keys.head()

,GeneID,Localization
0,G234062,mitochondria
1,G234081,nucleus
2,G234086,mitochondria
3,G234087,nucleus
4,G234094,nucleus


# Model Evaluation

In [168]:
def find_accuracy(predictions, id_attribute, keys):
    '''
    Input
        predictions: the classes of the test data that my model found
        keys: the true classes of the test data
    '''
    local_predictions = predictions.set_index(id_attribute, drop=True)
    # This will automatically match GeneID since 'GeneID' is index for both dataframes
    combined_dataframe = local_predictions.join(keys.set_index(id_attribute, drop=True))

    n = combined_dataframe.index.size
    # Count number of correctly classified tuples
    number_correct = 0
    for index, row in combined_dataframe.iterrows():
        if row['Prediction'] == row['Localization']:
            number_correct = number_correct + 1

    return number_correct / n

In [230]:
find_accuracy(predictions=my_model.predictions, id_attribute='GeneID', keys=new_keys)

0.57

# Write Results to Output File

In [233]:
my_model.predictions.to_csv(path_or_buf='ModelPredictions.csv', header=False, index=False)